# everything works except there is no member dimension soooooo i don't know where they are!!


In [1]:
import xarray as xr, matplotlib.pyplot as plt
from importlib import reload # need to use this if I edit a function file
import os
import numpy as np
import pandas as pd
import cartopy.crs as ccrs # to add in continents and change map projections 
from matplotlib.colors import LinearSegmentedColormap # to change colour bar????
import dask.diagnostics # dask allows you to check how long something is taking to load
import climtas # needed to count event statistics with a specified duration

In [2]:
# import custom functions
import sys 
sys.path.append('/home/563/kb6999/Functions') # use this if the function file is in a different directory to the notebook

import frequently_used_functions as func
import plotting_functions as fplot
import model_functions as funcM
import reanalysis_functions as funcR

In [3]:
# Create a list with all the strings identifying your members
# use a combination of range and zfill to get the leading zeros.
member_list = [str(x).zfill(2) for x in range(1,81)]

In [25]:
def add_member(ds):
    ds.coords['member'] = ds.attrs['history']
    return ds

In [26]:
path = f'/g/data/ua8/C20C/v3/member_monthly/TMPS/*/*_mem0*.nc'
member_ds_t = xr.open_mfdataset(path, parallel=True, preprocess=add_member).chunk(chunks={'time':-1, 'lat':100, 'lon':100})
member_ds_t

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 182 times more chunks
  return self.array[key]
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 182 times more chunks
  return self.array[key]


<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 512, lon: 1023, time: 1992)
Coordinates:
  * lat        (lat) float64 -89.46 -89.46 -88.77 -88.77 ... 88.77 89.46 89.46
  * lon        (lon) float64 0.0 0.703 0.7031 1.406 ... 358.5 358.6 359.2 359.3
  * time       (time) datetime64[ns] 1850-01-16T10:30:00 ... 2015-12-16T10:30:00
    member     (time, lat) <U371 dask.array<chunksize=(1992, 100), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (lon, lat, time, bnds) datetime64[ns] dask.array<chunksize=(100, 100, 1992, 2), meta=np.ndarray>
    TMP        (time, lat, lon) float32 dask.array<chunksize=(1992, 100, 100), meta=np.ndarray>
Attributes: (12/13)
    CDI:            Climate Data Interface version 1.9.3 (http://mpimet.mpg.d...
    history:        Sun Dec 23 02:29:36 2018: cdo -v -f nc4 -r -monavg TMPS.1...
    institution:    NOAA ESRL Physical Sciences Division & CU/CIRES \n
    Conventions:    CF-1.6
    platform:       Model
    citation:       <http://onlinelibrary.wiley.com/doi/10.1002/qj.776/abstra...
    ...             ...
    observations:   International Surface Pressure Databank version 4.7
    frequency:      mon
    CDO:            Climate Data Operators version 1.9.3 (http://mpimet.mpg.d...
    comments:       Data is from \n NOAA-CIRES-DOE 20th Century Reanalysis ve...
    main_citation:  <https://doi.org/10.1002/qj.3598>.\n Slivinski et al. 201...
    home_page:      <https://go.usa.gov/XTd>

In [5]:
# load temp data
path = f'/g/data/ua8/C20C/v3/member_monthly/TMPS/*/*_mem0*.nc'
member_ds_t = xr.open_mfdataset(path, parallel=True).chunk(chunks={'time':-1, 'lat':100, 'lon':100})
member_ds_t

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 182 times more chunks
  return self.array[key]
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 182 times more chunks
  return self.array[key]


<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 512, lon: 1023, time: 1992)
Coordinates:
  * lat        (lat) float64 -89.46 -89.46 -88.77 -88.77 ... 88.77 89.46 89.46
  * lon        (lon) float64 0.0 0.703 0.7031 1.406 ... 358.5 358.6 359.2 359.3
  * time       (time) datetime64[ns] 1850-01-16T10:30:00 ... 2015-12-16T10:30:00
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (lon, lat, time, bnds) datetime64[ns] dask.array<chunksize=(100, 100, 1992, 2), meta=np.ndarray>
    TMP        (time, lat, lon) float32 dask.array<chunksize=(1992, 100, 100), meta=np.ndarray>
Attributes: (12/13)
    CDI:            Climate Data Interface version 1.9.3 (http://mpimet.mpg.d...
    history:        Sun Dec 23 02:29:36 2018: cdo -v -f nc4 -r -monavg TMPS.1...
    institution:    NOAA ESRL Physical Sciences Division & CU/CIRES \n
    Conventions:    CF-1.6
    platform:       Model
    citation:       <http://onlinelibrary.wiley.com/doi/10.1002/qj.776/abstra...
    ...             ...
    observations:   International Surface Pressure Databank version 4.7
    frequency:      mon
    CDO:            Climate Data Operators version 1.9.3 (http://mpimet.mpg.d...
    comments:       Data is from \n NOAA-CIRES-DOE 20th Century Reanalysis ve...
    main_citation:  <https://doi.org/10.1002/qj.3598>.\n Slivinski et al. 201...
    home_page:      <https://go.usa.gov/XTd>

In [6]:
# rename the tmp variable
member_ds_t = member_ds_t.rename({'TMP':'tmp'})

In [11]:
member_ds_t

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 512, lon: 1023, time: 1992)
Coordinates:
  * lat        (lat) float64 -89.46 -89.46 -88.77 -88.77 ... 88.77 89.46 89.46
  * lon        (lon) float64 0.0 0.703 0.7031 1.406 ... 358.5 358.6 359.2 359.3
  * time       (time) datetime64[ns] 1850-01-16T10:30:00 ... 2015-12-16T10:30:00
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (lon, lat, time, bnds) datetime64[ns] dask.array<chunksize=(100, 100, 1992, 2), meta=np.ndarray>
    tmp        (time, lat, lon) float32 dask.array<chunksize=(1992, 100, 100), meta=np.ndarray>
Attributes: (12/13)
    CDI:            Climate Data Interface version 1.9.3 (http://mpimet.mpg.d...
    history:        Sun Dec 23 02:29:36 2018: cdo -v -f nc4 -r -monavg TMPS.1...
    institution:    NOAA ESRL Physical Sciences Division & CU/CIRES \n
    Conventions:    CF-1.6
    platform:       Model
    citation:       <http://onlinelibrary.wiley.com/doi/10.1002/qj.776/abstra...
    ...             ...
    observations:   International Surface Pressure Databank version 4.7
    frequency:      mon
    CDO:            Climate Data Operators version 1.9.3 (http://mpimet.mpg.d...
    comments:       Data is from \n NOAA-CIRES-DOE 20th Century Reanalysis ve...
    main_citation:  <https://doi.org/10.1002/qj.3598>.\n Slivinski et al. 201...
    home_page:      <https://go.usa.gov/XTd>

## area weighting

In [16]:
# area weighting and convert to np array using .tmp
reanal_w = member_ds_t.tmp*np.cos(member_ds_t.lat*(np.pi/180))

In [17]:
# group the data by month (take the monthly mean)
reanal_mon = reanal_w.resample(time='M').mean(dim='time')

# calculate monthly anomalies

In [19]:
# calulate anomalies
monthly_anom_glob = func.monthly_anomaly(reanal_mon, '1850-01', '1881-01')

In [20]:
monthly_anom_glob

,Array,Chunk
Bytes,8.35 GB,80.00 kB
Shape,"(1992, 512, 1023)","(1, 100, 100)"
Count,1323257 Tasks,131472 Chunks
Type,float64,numpy.ndarray


In [ ]:
monthly_anom_glob.to_netcdf(f'/g/data/w48/kb6999/20CR_data_netcdfs/R_mon_anom_Glob_tmp.nc')

In [ ]:
# # import landmask dataset 
# landfrac_ds = xr.open_dataset('/g/data/w48/kb6999/20CR_data_netcdfs/land_20CR.nc')
# landmask = landfrac_ds